# Nesting MicroHH in ERA5

This notebook contains documentation and examples for nesting MicroHH with open boundary conditions in ERA5.

Nesting LES-in-LES is documented separately in `TODO.ipynb`.

In [ ]:
#%matplotlib ipympl

# Standard library
import logging
import glob
import os

# Third-party.
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import ls2d
import cartopy.crs as ccrs
import nest_asyncio
import xarray as xr

# Local library
from microhhpy.spatial import Domain, plot_domains
from microhhpy.real import create_input_from_regular_latlon
from microhhpy.thermo import calc_moist_basestate

TF = np.float64

# Needed to use asyncio with Jupyter notebooks.``
nest_asyncio.apply()

# Set output level `microhhpy`.
logger = logging.getLogger("microhhpy")
logger.setLevel(logging.DEBUG)   # or DEBUG

## ERA5 data

For now, we use (LS)²D to download and read the ERA5 data. This can be simplified later, as we only need a few 3D fields from ERA5, and do not use the large-scale forcings typically required for a doubly periodic LES.

In [ ]:
settings = {
    'start_date'  : datetime(year=2022, month=4, day=1, hour=8),
    'end_date'    : datetime(year=2022, month=4, day=1, hour=9),
    'central_lon' : 4.8,
    'central_lat' : 53,
    'area_size'   : 5,
    'case_name'   : 'slocs_rf',
    'era5_path'   : '/home/scratch1/bart/LS2D_ERA5/',
    'era5_expver' : 1,
    'cdsapirc'    : '/home/bart/.cdsapirc_ads'
    }

era5 = ls2d.Read_era5(settings)
era5.calculate_forcings(n_av=3, method='2nd')

## Spatial projection

To nest LES in ERA5, a transformation is needed from the LES grid (in meters) to geographic coordinates (latitude / longitude in degrees).

The `Domain()` class provides a simple way to define the domain. The spatial transformation is performed using `pyproj`, based on the `proj_str` definition.

In [ ]:
lon = 4.8
lat = 53.
proj_str = f'+proj=lcc +lat_1={lat-1} +lat_2={lat+1} +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

dom0 = Domain(
    xsize=256_000,
    ysize=256_000,
    itot=128,
    jtot=128,
    n_ghost=3,
    n_sponge=5,
    lon=lon,
    lat=lat,
    anchor='center',
    proj_str=proj_str
    )

plot_domains([dom0], use_projection=True)

Note that each `Domain` instance includes two projections: one with ghost cells and one without. The version with ghost cells is needed for generating the lateral boundary conditions.

The padded projection actually includes `nghost + 1` ghost cells. The extra cells are needed to create divergence free fields in the east- and north-most ghost cells.

In [ ]:
print(dom0.proj.itot, dom0.proj_pad.itot)

Coordinate pairs are available for each grid point location `(u, v, scalar)`:

In [ ]:
plt.figure(figsize=(4,4))

plt.scatter(dom0.proj.lon,   dom0.proj.lat,   marker='o', facecolor='none', edgecolor='C0', label='scalar')
plt.scatter(dom0.proj.lon_u, dom0.proj.lat_u, marker='>', facecolor='none', edgecolor='C1', label='u')
plt.scatter(dom0.proj.lon_v, dom0.proj.lat_v, marker='^', facecolor='none', edgecolor='C2', label='v')

plt.xlim(dom0.proj.central_lon-0.03, dom0.proj.central_lon+0.03)
plt.ylim(dom0.proj.central_lat-0.03, dom0.proj.central_lat+0.03)

plt.legend()

## Vertical grid

In [ ]:
zsize = 3200
ktot = 128
dz = zsize / ktot
z = np.arange(dz/2, zsize, dz)

## Basestate density

Interpolate mean ERA5 fields to LES grid as basis for the LES basestate density. Again, this has to _perfectly_ match the basestate density used by MicroHH.

In [ ]:
les_input = era5.get_les_input(z)

bs = calc_moist_basestate(
    les_input['thl'][0,:].values,
    les_input['qt'][0,:].values,
    float(les_input['ps'][0].values),
    z,
    zsize,
    dtype=TF)

<div class="alert alert-block alert-info">NOTE: using an incorrect vertical grid definition or a base state density that doesn't match the one in MicroHH is one of the easiest ways to mess up the initialization. If your MicroHH simulations are not divergence-free at `t==0`, this should be the first thing to check.</div>

## Initial conditions

The initial conditions are (not so...) simply the 3D ERA5 fields tri-linearly interpolated to the LES grid.

To reduce the blocky structures that result from interpolating coarse ERA5 data, a Gaussian filter with standard deviation `sigma_h` is applied after interpolation.

### Momentum & divergence
The momentum fields require special treatment because they must be divergence-free. To achieve this, the `(u, v)` velocity components are corrected such that the resulting velocity field is divergence free, while preserving the original vertical velocity from the large-scale model.

### Output

The initial conditions are written directly as binary input files (e.g. `u.0000000`) for MicroHH in the specified `output_dir`. By setting a `name_suffix`, the files can optionally be written with a custom name, such as `u_some_name.0000000`. This allows you to overwrite the homogeneous 3D restart files generated during the `init` phase of MicroHH with the fields derived from ERA5.

<div class="alert alert-block alert-info">NOTE: When using (LS)²D data, be sure to use `wls` (and not `w`) for the vertical velocity!</div>

In [ ]:
fields_era = {
    'u': era5.u[:,:,:,:],
    'v': era5.v[:,:,:,:],
    'w': era5.wls[:,:,:,:],
    'thl': era5.thl[:,:,:,:],
    'qt': era5.qt[:,:,:,:],
}

# Gaussian filter size (m) of filter after interpolation.
sigma_h = 10_000

# 3D buffer.
zstart_buffer = 0.75 * zsize

ds = create_input_from_regular_latlon(
    fields_era,
    era5.lons.data,   # Strip off array masks.
    era5.lats.data,
    era5.z[:,:,:,:],
    era5.p[:,:,:,:],
    era5.time_sec,
    z,
    zsize,
    zstart_buffer,
    bs['rho'],
    bs['rhoh'],
    dom0,
    sigma_h,
    perturb_size=4,
    perturb_amplitude={'thl': 0.1, 'qt':0.1e-3},
    perturb_max_height=1000,
    clip_at_zero=['qt'],
    name_suffix='era5',
    output_dir='.',
    ntasks=16,
    save_netcdf=True,
    float_type=TF)

Quick plot of potential temperature field:

In [ ]:
def plot_with_bounds(x, y, fld, vmin, vmax):
    """
    pcolormesh `fld` with bounds plotted on top.
    """
    dx = x[1] - x[0]
    dy = y[1] - y[0]

    x0 = x[0] - dx/2.
    x1 = x[-1] + dx/2.

    y0 = y[0] - dy/2.
    y1 = y[-1] + dy/2.

    plt.pcolormesh(x, y, fld, vmin=vmin, vmax=vmax)
    plt.plot([x0, x1, x1, x0, x0], [y0, y0, y1, y1, y0], 'w:')

 
thl = np.fromfile('thl_era5.0000000', dtype=TF)
thl = thl.reshape((ktot, dom0.jtot, dom0.itot))

lbc = xr.open_dataset('lbc_ds.nc', decode_times=False)

k = 0

vmin = thl[k,:,:].min()
vmax = thl[k,:,:].max()

plt.figure()
plot_with_bounds(dom0.x, dom0.y, thl[k,:,:], vmin=vmin, vmax=vmax)
plot_with_bounds(dom0.x, dom0.y, thl[k,:,:], vmin=vmin, vmax=vmax)
plot_with_bounds(lbc.xgw, lbc.y, lbc.thl_west[0,k,:,:], vmin=vmin, vmax=vmax)
plot_with_bounds(lbc.xge, lbc.y, lbc.thl_east[0,k,:,:], vmin=vmin, vmax=vmax)
plot_with_bounds(lbc.x, lbc.ygn, lbc.thl_north[0,k,:,:], vmin=vmin, vmax=vmax)
plot_with_bounds(lbc.x, lbc.ygs, lbc.thl_south[0,k,:,:], vmin=vmin, vmax=vmax)

#plot_domains([dom0], use_projection=True)
#plt.pcolormesh(dom0.proj.lon, dom0.proj.lat, thl[0,:,:], transform=ccrs.PlateCarree())
#plt.colorbar()

## Cleanup!

In [ ]:
files = glob.glob('*00*')
for f in files:
    os.remove(f)